In [ ]:
#Dependencies
import pandas as pd
import numpy as np


In [ ]:
#Extracting Files (Kdrama)
kdrama_file = "./Data/imdb.csv"
kdrama_df = pd.read_csv(kdrama_file)
kdrama_df.head()

In [ ]:
#Cleaning Files (kdrama)
cleaned_kdrama_df = kdrama_df[["kdrama_name", "imdb_rating"]].copy()
cleaned_kdrama_df.sort_values(by=['imdb_rating'], inplace=True, ascending=False)
cleaned_kdrama_df.head()

In [ ]:
#Genres
kdrama_genre_file = "./Data/genres.csv"
kdrama_genre_file_df = pd.read_csv(kdrama_genre_file)
kdrama_genre_file_df.head()

In [ ]:
kdrama_joined= pd.merge(cleaned_kdrama_df, kdrama_genre_file_df, on='kdrama_name', how='inner')
kdrama_joined.head()

In [ ]:
kdrama_joined["genre"] = kdrama_joined["genre"].str.upper()
kdrama_joined.head()

In [ ]:
kdrama_joined_genre_df = kdrama_joined.groupby(['kdrama_name'])['genre'].apply(lambda x : np.unique(list(x))).reset_index()


In [ ]:
kdrama_joined_genre_df.head()

In [ ]:
kdrama_joined = kdrama_joined[["kdrama_name", "imdb_rating"]]
kdrama_joined.head()


In [ ]:
ratings_genre_combined = pd.merge(kdrama_joined, kdrama_joined_genre_df, on='kdrama_name', how='inner')
final_df = ratings_genre_combined.drop_duplicates(subset="kdrama_name")
final_df.head()

In [ ]:
final_df = final_df[final_df['imdb_rating'] > 8] 
final_df

In [ ]:
final_df = final_df.drop([0, 1])
final_df = final_df.reset_index()
